In [ ]:
import numpy as np
import nibabel as nib
import glob
import os
from tifffile import imsave
from PIL import Image
import cv2

In [ ]:
#Extracción de imágenes de 512x512 pixeles de los archivos .nii

#Condiciones:
  #Cantidad de imágenes igual a la de máscaras
  #Cantidad de valores únicos igual a 3 ([0,1,2])
  #Solo se consideran los casos donde hay más de 10 pixeles de interés (cálculos renales)
np.seterr(divide='ignore', invalid='ignore') #Para ignorar la advertencia de dividir entre cero
for a in range(260):
    #a: número de tomografía computarizada
    #num: String que permite identificar al archivo .nii correspondiente
    if a < 10:
        num = '00'+str(a)
    elif a < 100:
        num = '0'+str(a)
    else:
        num = str(a)
    #Lectura de los archivos .nii (Dataset)
    kidney_images=nib.load('D:\DATASETS\Kidney dataset\Zenodo\parts\wd0'+num+'_ps_image.nii').get_fdata()
    kidney_masks=nib.load('D:\DATASETS\Kidney dataset\Zenodo\parts\wd0'+num+'_ps_mask.nii').get_fdata()
    aux_masks=kidney_masks.astype(np.uint8) #aux_masks: Auxiliar para determinar valores mínimos
    max1 = kidney_images.shape[2] #Cantidad de imágenes
    max2 = kidney_masks.shape[2] #Cantidad de máscaras
    #Se comprueba si la cantidad de imágenes = cantidad de máscaras y si estas últimas tienen 3 valores únicos
    if(max1==max2 and len(np.unique(aux_masks))==3):
        #Bucle para determinar el índice de la máscara
        for i in range(max1):
            count=0 #Cantidad de pixeles de cálculos renales
            #Bucle dentro de los pixeles de una máscaras
            for j in range(512):
                for k in range(512):
                    #Verificar si el pixel en cuestión pertenece a un cálculo renal
                    if(kidney_masks[j,k,i]==1):
                        count+=1
            if(count>9):
                #Máscara que cumple las condiciones previas
                final_mask = kidney_masks[:,:,i]
                #Se guarda la máscara en formato .png para que la imagen no pierda calidad
                cv2.imwrite('D:\DATASETS\Kidney dataset\Zenodo\kidStn512_aux\masks\ct'+num+'_mask'+str(i)+'.png', final_mask)
                #Imagen que cumple las condiciones previas y corresponde a la máscara
                final_image = kidney_images[:,:,i]
                #Se guarda la imagen en formato .jpg
                cv2.imwrite('D:\DATASETS\Kidney dataset\Zenodo\kidStn512_aux\images\ct'+num+'_image'+str(i)+'.jpg', final_image)
print('Done')